<a href="https://colab.research.google.com/github/inregg/colab/blob/master/Copy_of_more_DOTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [38]:
#!pip install bayesian-optimization
!pip install catboost
import os
#import ujson as json
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import ShuffleSplit, KFold, train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
import lightgbm 
from xgboost import XGBClassifier
#from bayes_opt import BayesianOptimization
import warnings
from collections import defaultdict
from tqdm import tqdm_notebook
import collections
from scipy.sparse import hstack
import catboost as ctb

     |████████████████████████████████| 62.9MB 115kB/s 


In [0]:
train = pd.read_csv('/content/gdrive/My Drive/dota/train_new_feat.csv', index_col='match_id_hash')
target = pd.read_csv('/content/gdrive/My Drive/dota/train_targets.csv', index_col='match_id_hash')
test = pd.read_csv('/content/gdrive/My Drive/dota/test_new_feat.csv', index_col='match_id_hash')

In [0]:
y_train = target['radiant_win']

In [0]:
r_heroes_train = train[[f'r{i}_hero_id' for i in range(1, 6)]].apply(lambda row: 
                                                     ' '.join([str(i) for i in row.values if i!=0]), axis=1).tolist()
d_heroes_train = train[[f'd{i}_hero_id' for i in range(1, 6)]].apply(lambda row: 
                                                     ' '.join([str(i) for i in row.values if i!=0]), axis=1).tolist()

r_heroes_test = test[[f'r{i}_hero_id' for i in range(1, 6)]].apply(lambda row: 
                                                     ' '.join([str(i) for i in row.values if i!=0]), axis=1).tolist()
d_heroes_test = test[[f'd{i}_hero_id' for i in range(1, 6)]].apply(lambda row: 
                                                     ' '.join([str(i) for i in row.values if i!=0]), axis=1).tolist()                                                     
vecorizer = CountVectorizer(tokenizer = lambda s: s.split(), 
                            ngram_range  = (1,3),
                            max_features = 120)
r_train_vec = vecorizer.fit_transform(r_heroes_train)
d_train_vec = vecorizer.transform(d_heroes_train)
r_test_vec = vecorizer.transform(r_heroes_test)
d_test_vec = vecorizer.transform(d_heroes_test)

In [0]:
train_heroes = train[[f'{t}{i}_hero_id' for t in ['r', 'd'] for i in range(1, 6)]].apply(lambda row: ' '.join([str(i) for i in row.values]), axis=1).str.split()
test_heroes = test[[f'{t}{i}_hero_id' for t in ['r', 'd'] for i in range(1, 6)]].apply(lambda row: ' '.join([str(i) for i in row.values]), axis=1).str.split()

In [0]:
win_count = defaultdict(int)
lose_count = defaultdict(int)
for i in range(len(target)):
  if y_train[i]==True:
    for j in range(5):
      win_count[train_heroes[i][j]]+=1
      lose_count[train_heroes[i][j+5]]+=1
  else :
    for k in range(5):
      win_count[train_heroes[i][k+5]]+=1
      lose_count[train_heroes[i][k]]+=1

In [0]:
winrate = {v:(win_count[v]/(win_count[v]+lose_count[v])) for v ,k in  win_count.items()}
train_wr = train_heroes.apply(lambda row: [winrate[row[i]] for i in range(10)])
train['r_wr_mean'] = train_wr.apply(lambda row: np.array([row[i] for i in range(5)]).mean())
train['r_wr_std'] = train_wr.apply(lambda row: np.array([row[i] for i in range(5)]).std())
train['d_wr_mean'] = train_wr.apply(lambda row: np.array([row[i] for i in range(5, 10)]).mean())
train['d_wr_std'] = train_wr.apply(lambda row: np.array([row[i] for i in range(5, 10)]).std())
train['wr_ratio'] = train['r_wr_mean']/train['d_wr_mean']
test_wr = test_heroes.apply(lambda row: [winrate[row[i]] for i in range(10)])
test['r_wr_mean'] = test_wr.apply(lambda row: np.array([row[i] for i in range(5)]).mean())
test['r_wr_std'] = test_wr.apply(lambda row: np.array([row[i] for i in range(5)]).std())
test['d_wr_mean'] = test_wr.apply(lambda row: np.array([row[i] for i in range(5, 10)]).mean())
test['d_wr_std'] = test_wr.apply(lambda row: np.array([row[i] for i in range(5, 10)]).std())
test['wr_ratio'] = test['r_wr_mean']/test['d_wr_mean']

In [0]:
def get_heroes_combo (heroes):
  sheroes = np.sort(heroes)
  return np.concatenate((sheroes[0]*1000+sheroes[1:5],sheroes[1]*1000+sheroes[2:5], sheroes[2]*1000+sheroes[3:5], sheroes[3]*1000+sheroes[4:5]), axis = None)

In [0]:
r_hero_combo = train[[f'r{i}_hero_id' for i in range(1, 6)]].apply(lambda row: get_heroes_combo(row.values), axis = 1).values
d_hero_combo = train[[f'd{i}_hero_id' for i in range(1, 6)]].apply(lambda row: get_heroes_combo(row.values), axis = 1).values
r_hero_combo_test = test[[f'r{i}_hero_id' for i in range(1, 6)]].apply(lambda row: get_heroes_combo(row.values), axis = 1).values
d_hero_combo_test = test[[f'd{i}_hero_id' for i in range(1, 6)]].apply(lambda row: get_heroes_combo(row.values), axis = 1).values

In [0]:
win_count_combo = defaultdict(int)
lose_count_combo = defaultdict(int)
for i in range(len(target)):
  if y_train[i]==True:
    for j in range (10):
      win_count_combo[str(r_hero_combo[i][j])]+=1
      lose_count_combo[str(d_hero_combo[i][j])]+=1
  else:
    for j in range (10):
      win_count_combo[str(d_hero_combo[i][j])]+=1
      lose_count_combo[str(r_hero_combo[i][j])]+=1

In [0]:
combo_wr = {v:(win_count_combo[v]/(win_count_combo[v]+lose_count_combo[v])) for v ,k in  win_count_combo.items()}
r_wr_combo = [[combo_wr[str(i)] for i in row] for row in r_hero_combo]
d_wr_combo = [[combo_wr[str(i)] for i in row] for row in d_hero_combo]
r_wr_combo_test = [[combo_wr[str(i)] for i in row] for row in r_hero_combo_test]
d_wr_combo_test = [[combo_wr[str(i)] for i in row] for row in d_hero_combo_test]
train['r_wr_combo_mean'] = [np.mean(row) for row in r_wr_combo]
train['d_wr_combo_mean'] = [np.mean(row) for row in d_wr_combo]
train['r_wr_combo_std'] = [np.std(row) for row in r_wr_combo]
train['d_wr_combo_std'] = [np.std(row) for row in d_wr_combo]
train['combo_wr_ratio'] = train['r_wr_combo_mean']/train['d_wr_combo_mean']
test['r_wr_combo_mean'] = [np.mean(row) for row in r_wr_combo_test]
test['d_wr_combo_mean'] = [np.mean(row) for row in d_wr_combo_test]
test['r_wr_combo_std'] = [np.std(row) for row in r_wr_combo_test]
test['d_wr_combo_std'] = [np.std(row) for row in d_wr_combo_test]
test['combo_wr_ratio'] = test['r_wr_combo_mean']/test['d_wr_combo_mean']

In [0]:
for c in ['kills', 'deaths', 'assists', 'denies', 'gold', 'lh', 'xp', 'health', 'max_health', 'max_mana', 'level', 'x', 'y', 'stuns', 'creeps_stacked', 'camps_stacked', 'rune_pickups',
          'firstblood_claimed', 'teamfight_participation', 'towers_killed', 'roshans_killed', 'obs_placed', 'sen_placed', 'ability_level', 'max_hero_hit', 'purchase_count',
          'count_ability_use', 'damage_dealt', 'damage_received']:
    r_columns = [f'r{i}_{c}' for i in range(1, 6)]
    d_columns = [f'd{i}_{c}' for i in range(1, 6)]
    
    train['r_total_' + c] = train[r_columns].sum(1)
    train['d_total_' + c] = train[d_columns].sum(1)
    train['total_' + c + '_ratio'] = train['r_total_' + c] / train['d_total_' + c]
    
    test['r_total_' + c] = test[r_columns].sum(1)
    test['d_total_' + c] = test[d_columns].sum(1)
    test['total_' + c + '_ratio'] = test['r_total_' + c] / test['d_total_' + c]
    
    train['r_std_' + c] = train[r_columns].std(1)
    train['d_std_' + c] = train[d_columns].std(1)
    train['std_' + c + '_ratio'] = train['r_std_' + c] / train['d_std_' + c]
    
    test['r_std_' + c] = test[r_columns].std(1)
    test['d_std_' + c] = test[d_columns].std(1)
    test['std_' + c + '_ratio'] = test['r_std_' + c] / test['d_std_' + c]
    
    train['r_mean_' + c] = train[r_columns].mean(1)
    train['d_mean_' + c] = train[d_columns].mean(1)
    train['mean_' + c + '_ratio'] = train['r_mean_' + c] / train['d_mean_' + c]
    
    test['r_mean_' + c] = test[r_columns].mean(1)
    test['d_mean_' + c] = test[d_columns].mean(1)
    test['mean_' + c + '_ratio'] = test['r_mean_' + c] / test['d_mean_' + c]

In [0]:
X_train = hstack([r_train_vec, d_train_vec, train.values])
X_test  = hstack([r_test_vec, d_test_vec, test.values])

In [0]:
def lgb_eval(num_leaves,max_depth,lambda_l2,lambda_l1,min_child_samples, min_data_in_leaf):
    params = {
    "objective" : "binary",
    "metric" : "auc", 
    'is_unbalance': True,
    "num_leaves" : int(num_leaves),
    "max_depth" : int(max_depth),
    "lambda_l2" : lambda_l2,
    "lambda_l1" : lambda_l1,
    "num_threads" : 20,
    "min_child_samples" : int(min_child_samples),
    'min_data_in_leaf': int(min_data_in_leaf),
    "learning_rate" : 0.03,
    "subsample_freq" : 5,
    "bagging_seed" : 42,
    "verbosity" : -1
    }
    
    lgtrain = lightgbm.Dataset(X_train, y_train,
#                               categorical_feature=categorical_features
                               )
    cv_result = lightgbm.cv(params,
                       lgtrain,
                       10000,
                       early_stopping_rounds=300,
                       stratified=True,
                       nfold=5)
    return cv_result['auc-mean'][-1]

def lgb_train(num_leaves,max_depth,lambda_l2,lambda_l1,min_child_samples, min_data_in_leaf):
    params = {
    "objective" : "binary",
    "metric" : "auc", 
    'is_unbalance': True,
    "num_leaves" : int(num_leaves),
    "max_depth" : int(max_depth),
    "lambda_l2" : lambda_l2,
    "lambda_l1" : lambda_l1,
    "num_threads" : 20,
    "min_child_samples" : int(min_child_samples),
    'min_data_in_leaf': int(min_data_in_leaf),
    "learning_rate" : 0.03,
    "subsample_freq" : 5,
    "bagging_seed" : 42,
    "verbosity" : -1
    }
    t_x,v_x,t_y,v_y = train_test_split(X_train, y_train,test_size=0.2)
    lgtrain = lightgbm.Dataset(t_x, t_y,
#                               categorical_feature=categorical_features
                               )
    lgvalid = lightgbm.Dataset(v_x, v_y,
#                               categorical_feature=categorical_features
                               )
    model = lightgbm.train(params, lgtrain, 4000, valid_sets=[lgvalid], early_stopping_rounds=400, verbose_eval=200)
    pred_test_y = model.predict(X_test, num_iteration=model.best_iteration)
    return pred_test_y, model
    
def param_tuning(init_points,num_iter,**args):
    lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (15, 200),
                                                'max_depth': (5, 63),
                                                'lambda_l2': (0.0, 5),
                                                'lambda_l1': (0.0, 5),
                                                'min_child_samples': (50, 5000),
                                                'min_data_in_leaf': (50, 300)
                                                })

    lgbBO.maximize(init_points=init_points, n_iter=num_iter,**args)
    return lgbBO

In [0]:
categorical_features= ['lobby_type', 'game_mode', 'r1_hero_id', 'r2_hero_id', 'r3_hero_id', 'r4_hero_id', 'r5_hero_id',
                       'd1_hero_id', 'd2_hero_id', 'd3_hero_id', 'd4_hero_id', 'd5_hero_id']

In [0]:
lgb_eval(**params)
#0.8427974187036472
#0.8438491500041467

0.8438491500041467

In [0]:
result = param_tuning(10,50)

In [0]:
params = result.max['params']
params

{'lambda_l1': 4.665580784170334,
 'lambda_l2': 1.2802911269689954,
 'max_depth': 60.048596284780366,
 'min_child_samples': 4915.273285235697,
 'min_data_in_leaf': 296.32964773117334,
 'num_leaves': 16.295593155225685}

In [0]:
pred_test_y1, _ = lgb_train(**params)
pred_test_y2, _ = lgb_train(**params)
pred_test_y3, _ = lgb_train(**params)
y_pred = (pred_test_y1 + pred_test_y2 + pred_test_y3)/3

Training until validation scores don't improve for 400 rounds.
[200]	valid_0's auc: 0.840637
[400]	valid_0's auc: 0.841751
[600]	valid_0's auc: 0.841646
[800]	valid_0's auc: 0.841107
Early stopping, best iteration is:
[493]	valid_0's auc: 0.842004


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:478: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


Training until validation scores don't improve for 400 rounds.
[200]	valid_0's auc: 0.843122
[400]	valid_0's auc: 0.844646
[600]	valid_0's auc: 0.845011
[800]	valid_0's auc: 0.845346
[1000]	valid_0's auc: 0.845075
Early stopping, best iteration is:
[794]	valid_0's auc: 0.845383
Training until validation scores don't improve for 400 rounds.
[200]	valid_0's auc: 0.839622
[400]	valid_0's auc: 0.840523
[600]	valid_0's auc: 0.840524
[800]	valid_0's auc: 0.839789
Early stopping, best iteration is:
[452]	valid_0's auc: 0.840727


In [0]:
df_submission_extended = pd.DataFrame(
    {'radiant_win_prob': y_pred}, 
    index=test.index)
df_submission_extended.to_csv('submission_lgb_bag_tuned.csv')

In [0]:
def ctb_eval (l2_leaf_reg, max_depth, min_data_in_leaf, learning_rate):
  params = {
    'loss_function': 'Logloss',
    'iterations': 4000,
    'eval_metric': 'AUC',
    'early_stopping_rounds': 100,
    'task_type': 'GPU',
    'l2_leaf_reg': l2_leaf_reg,
    'max_depth': round(max_depth),
    'min_data_in_leaf': round(min_data_in_leaf),
    'learning_rate': learning_rate
  }
  ctb_pool = ctb.Pool(data = train, label=y_train.astype(int).values, cat_features=cat_features)
  cv_data = ctb.cv(
    params = params,
    pool = ctb_pool,
    fold_count=5,
    shuffle=True,
    stratified=True,
    verbose=False
  )
  return cv_data['test-AUC-mean'].max()

def ctb_tuning (init_points,num_iter,**args):
  ctbBO = BayesianOptimization(ctb_eval, {'l2_leaf_reg': (0, 10),
                                          'max_depth': (5, 8),
                                          'min_data_in_leaf': (50, 300),
                                          'learning_rate':(0.01, 0.1)                                        
                                                })
  ctbBO.maximize(init_points=init_points, n_iter=num_iter,**args)
  return ctbBO
def ctb_pred(l2_leaf_reg, max_depth, min_data_in_leaf, learning_rate): 
    params = {
    'loss_function': 'Logloss',
    'iterations': 4000,
    'eval_metric': 'AUC',
    'early_stopping_rounds': 100,
    'task_type': 'GPU',
    'l2_leaf_reg': l2_leaf_reg,
    'max_depth': round(max_depth),
    'min_data_in_leaf': round(min_data_in_leaf),
    'learning_rate': learning_rate
    }
    t_x,v_x,t_y,v_y = train_test_split(train, y_train.astype(int).values,test_size=0.2)
    ctb_train_pool = ctb.Pool(data = t_x, label = t_y, cat_features=cat_features)
    ctb_eval_pool = ctb.Pool(data = v_x, label = v_y, cat_features=cat_features)
    ctb_clf = ctb.CatBoostClassifier(**params)
    ctb_clf.fit(ctb_train_pool, eval_set = ctb_eval_pool, early_stopping_rounds=100)
    return ctb_clf.predict_proba(test)[:, 1]

In [0]:
cat_features= ['lobby_type', 'game_mode', 'r1_hero_id', 'r2_hero_id', 'r3_hero_id', 'r4_hero_id', 'r5_hero_id',
                       'd1_hero_id', 'd2_hero_id', 'd3_hero_id', 'd4_hero_id', 'd5_hero_id']

In [0]:
ctb_result = ctb_tuning(10,20)

|   iter    |  target   | l2_lea... | learni... | max_depth | min_da... |
-------------------------------------------------------------------------
|  1        |  0.8439   |  0.7742   |  0.01793  |  5.386    |  158.1    |
|  2        |  0.8428   |  3.586    |  0.08014  |  5.628    |  291.5    |
|  3        |  0.8436   |  4.319    |  0.0705   |  5.239    |  90.99    |
|  4        |  0.8437   |  4.912    |  0.02995  |  6.787    |  276.8    |
|  5        |  0.8434   |  4.979    |  0.06742  |  5.214    |  247.4    |
|  6        |  0.8436   |  6.354    |  0.03847  |  7.345    |  251.8    |
|  7        |  0.8437   |  0.5541   |  0.03599  |  6.434    |  180.8    |
|  8        |  0.8438   |  3.443    |  0.03184  |  7.841    |  113.6    |
|  9        |  0.8438   |  2.648    |  0.0414   |  7.637    |  90.17    |
|  10       |  0.844    |  5.832    |  0.01503  |  7.63     |  100.1    |
|  11       |  0.8427   |  9.985    |  0.09057  |  7.547    |  50.14    |
|  12       |  0.8433   |  0.1086   | 

In [0]:
ctb_params = ctb_result.max['params']
ctb_params

In [0]:
ctb_params = {
              'l2_leaf_reg': 9.534,
              'max_depth': 7,
              'min_data_in_leaf': 350,
              'learning_rate':0.03
    
}

In [47]:
#0.843704617023468
score = ctb_eval(**ctb_params)
print(score)

0.8770698428153991


In [53]:
pred_test_y1 = ctb_pred(**ctb_params)
pred_test_y2 = ctb_pred(**ctb_params)
pred_test_y3 = ctb_pred(**ctb_params)
y_pred = (pred_test_y1 + pred_test_y2 + pred_test_y3)/3

0:	learn: 0.8329616	test: 0.8283516	best: 0.8283516 (0)	total: 335ms	remaining: 22m 20s
1:	learn: 0.8413425	test: 0.8386685	best: 0.8386685 (1)	total: 632ms	remaining: 21m 2s
2:	learn: 0.8444904	test: 0.8433184	best: 0.8433184 (2)	total: 898ms	remaining: 19m 57s
3:	learn: 0.8455553	test: 0.8449739	best: 0.8449739 (3)	total: 1.16s	remaining: 19m 17s
4:	learn: 0.8490763	test: 0.8473501	best: 0.8473501 (4)	total: 1.4s	remaining: 18m 41s
5:	learn: 0.8505740	test: 0.8490001	best: 0.8490001 (5)	total: 1.67s	remaining: 18m 30s
6:	learn: 0.8516139	test: 0.8503834	best: 0.8503834 (6)	total: 1.94s	remaining: 18m 27s
7:	learn: 0.8517103	test: 0.8501264	best: 0.8503834 (6)	total: 2.19s	remaining: 18m 13s
8:	learn: 0.8520265	test: 0.8509430	best: 0.8509430 (8)	total: 2.46s	remaining: 18m 12s
9:	learn: 0.8529411	test: 0.8518422	best: 0.8518422 (9)	total: 2.71s	remaining: 17m 59s
10:	learn: 0.8534006	test: 0.8523611	best: 0.8523611 (10)	total: 2.95s	remaining: 17m 49s
11:	learn: 0.8544309	test: 0.853

In [0]:
df_submission_extended = pd.DataFrame(
    {'radiant_win_prob': y_pred}, 
    index=test.index)
df_submission_extended.to_csv('submission_ctb_hero_sinergy.csv')

In [0]:
with open(os.path.join('/content/gdrive/My Drive/dota/', 'train_matches.jsonl')) as fin:
    # read the 18-th line
    for i in range(18):
        line = fin.readline()
    
    # read JSON into a Python object 
    match = json.loads(line)

In [0]:
def read_matches(matches_file):
    
    MATCHES_COUNT = {
        'test_matches.jsonl': 10000,
        'train_matches.jsonl': 39675,
    }
    _, filename = os.path.split(matches_file)
    total_matches = MATCHES_COUNT.get(filename)
    
    with open(matches_file) as fin:
        for line in tqdm_notebook(fin, total=total_matches):
            yield json.loads(line)

In [0]:
MATCH_FEATURES = [
    ('game_time', lambda m: m['game_time']),
    ('game_mode', lambda m: m['game_mode']),
    ('lobby_type', lambda m: m['lobby_type']),
    ('objectives_len', lambda m: len(m['objectives'])),
    ('chat_len', lambda m: len(m['chat'])),
]

PLAYER_FIELDS = [
    'hero_id',
    
    'kills',
    'deaths',
    'assists',
    'denies',
    
    'gold',
    'lh',
    'xp',
    'health',
    'max_health',
    'max_mana',
    'level',

    'x',
    'y',
    
    'stuns',
    'creeps_stacked',
    'camps_stacked',
    'rune_pickups',
    'firstblood_claimed',
    'teamfight_participation',
    'towers_killed',
    'roshans_killed',
    'obs_placed',
    'sen_placed',
]

def extract_features_csv(match):
    row = [
        ('match_id_hash', match['match_id_hash']),
    ]
    
    for field, f in MATCH_FEATURES:
        row.append((field, f(match)))
        
    for slot, player in enumerate(match['players']):
        if slot < 5:
            player_name = 'r%d' % (slot + 1)
        else:
            player_name = 'd%d' % (slot - 4)

        for field in PLAYER_FIELDS:
            column_name = '%s_%s' % (player_name, field)
            row.append((column_name, player[field]))
        row.append((f'{player_name}_ability_level', len(player['ability_upgrades'])))
        row.append((f'{player_name}_max_hero_hit', player['max_hero_hit']['value']))
        row.append((f'{player_name}_purchase_count', len(player['purchase_log'])))
        row.append((f'{player_name}_count_ability_use', sum(player['ability_uses'].values())))
        row.append((f'{player_name}_damage_dealt', sum(player['damage'].values())))
        row.append((f'{player_name}_damage_received', sum(player['damage_taken'].values())))
            
    return collections.OrderedDict(row)
    
def extract_targets_csv(match, targets):
    return collections.OrderedDict([('match_id_hash', match['match_id_hash'])] + [
        (field, targets[field])
        for field in ['game_time', 'radiant_win', 'duration', 'time_remaining', 'next_roshan_team']
    ])

In [0]:
PATH_TO_DATA = '/content/gdrive/My Drive/dota/'
df_new_features = []
df_new_targets = []

for match in read_matches(os.path.join(PATH_TO_DATA, 'train_matches.jsonl')):
    match_id_hash = match['match_id_hash']
    features = extract_features_csv(match)
    targets = extract_targets_csv(match, match['targets'])
    
    df_new_features.append(features)
    df_new_targets.append(targets)

In [0]:
df_new_features = pd.DataFrame.from_records(df_new_features).set_index('match_id_hash')
df_new_targets = pd.DataFrame.from_records(df_new_targets).set_index('match_id_hash')

In [0]:
test_new_features = []
for match in read_matches(os.path.join(PATH_TO_DATA, 'test_matches.jsonl')):
    match_id_hash = match['match_id_hash']
    features = extract_features_csv(match)
    
    test_new_features.append(features)
test_new_features = pd.DataFrame.from_records(test_new_features).set_index('match_id_hash')

In [0]:
df_new_features.to_csv('train_new_feat.csv')
df_new_targets.to_csv('target_new_feat.csv')
test_new_features.to_csv('test_new_feat.csv')